In [1]:
#import necessary dependecies
import plotly.express as px
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import os
import warnings
import numpy as np  
import seaborn as sns
import pandas as pd, os, gc
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, log_loss
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler, RobustScaler
%matplotlib inline
warnings.filterwarnings('ignore')
from typing import List

In [9]:
#Data Path

trains=pd.read_csv('Train.csv')
tests=pd.read_csv('Test.csv')
submissions=pd.read_csv('SampleSubmission.csv')

In [10]:
#print all dataset shape
print(trains.shape, tests.shape,submissions.shape)

(29132, 29) (10000, 29) (210000, 2)


In [12]:
train = trains
test = tests
submission = submissions

In [14]:
#Store Products ID in Multidimensional array
products =train[['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']]
products_t =test[['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']]

In [15]:
#Axis Transformation
train = train.melt(id_vars=train.columns[:8], value_vars=products, var_name = "PCODE", value_name="Label" )
test = test.melt(id_vars=test.columns[:8], value_vars=products, var_name = "PCODE", value_name="Label" )

In [16]:
#A merger
train['combiner']='x'

In [17]:
#Set
data=pd.concat([train,test],sort=False).reset_index(drop=True)

In [18]:
#preview random 3 sample
data.sample(3)

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,Label,combiner
671880,23D0OIU,1/10/2017,F,S,1968,XX25,RUFT,T4MS,SOP4,0,NaN
566239,NLRPX1J,1/8/2019,M,M,1980,1X1H,2A7I,T4MS,GHYX,0,x
404068,Z2W6FSU,1/8/2018,M,M,1967,748L,2A7I,T4MS,FM3X,0,x


In [19]:
#Confirm Distribution (TEST AND SAMPLE SUBMISSION FILE)
print('==' * 18);print('TRAIN LABEL DISTRIBUTION');print('==' * 18);print(train['Label'].value_counts())
print('==' * 18);print('TEST LABEL DISTRIBUTION');print('==' * 18);print(test['Label'].value_counts())
print('==' * 18);print('SAMPLE SUBMISSION LABEL DISTRIBUTION');print('==' * 18);print(submission['Label'].value_counts())

TRAIN LABEL DISTRIBUTION
0    545419
1     66353
Name: Label, dtype: int64
TEST LABEL DISTRIBUTION
0    197147
1     12853
Name: Label, dtype: int64
SAMPLE SUBMISSION LABEL DISTRIBUTION
0    197147
1     12853
Name: Label, dtype: int64


In [20]:
train=data[data.combiner.notnull()].reset_index(drop=True);test=data[data.combiner.isna()].reset_index(drop=True)
train.drop('combiner', inplace=True, axis=1);test.drop(['Label','combiner'], inplace=True, axis=1)
print(train.shape, test.shape, submission.shape)

(611772, 10) (210000, 9) (210000, 2)


In [21]:
#Concatenate new column towards submission requirement
train['ID X PCODE'] = train['ID'] + ' X ' + train['PCODE'];test['ID X PCODE'] = test['ID'] + ' X ' + test['PCODE']

In [22]:
#Re-arrange column (not necessary)
train=train[['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'PCODE','ID X PCODE',  'Label']]
test=test[['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'PCODE','ID X PCODE']]

In [23]:
#Lets Check for data co-existence in Train/Test
for col in ['PCODE','occupation_code', 'branch_code','occupation_category_code']:
    train_col = train[col].unique();test_col = test[col].unique()
    print ('Not in Test ' + col + ' ',[i  for i in train_col if i not in test_col])
    print ('Not in Train ' + col + ' ',[i  for i in test_col  if i not in train_col]);print('==' * 30)

Not in Test PCODE  []
Not in Train PCODE  []
Not in Test occupation_code  [&#39;IE90&#39;, &#39;Q0LY&#39;, &#39;JSAX&#39;, &#39;6XXU&#39;, &#39;8HRZ&#39;, &#39;OQMY&#39;, &#39;INEJ&#39;, &#39;VZN9&#39;, &#39;UC7E&#39;, &#39;PSUY&#39;, &#39;WSRG&#39;, &#39;JQH3&#39;, &#39;LGTN&#39;, &#39;738L&#39;, &#39;QQUP&#39;, &#39;2XZ1&#39;, &#39;CAAV&#39;, &#39;LLLH&#39;, &#39;W1X2&#39;, &#39;DHSN&#39;, &#39;IX8T&#39;, &#39;2US6&#39;, &#39;ZWPL&#39;, &#39;MEFQ&#39;, &#39;9B5B&#39;, &#39;JUIP&#39;, &#39;BFD1&#39;, &#39;A4ZC&#39;, &#39;IMHI&#39;, &#39;E5PF&#39;, &#39;GZA8&#39;, &#39;3YQ1&#39;, &#39;PJR4&#39;, &#39;NDL9&#39;, &#39;PPNK&#39;, &#39;2686&#39;, &#39;5LNN&#39;, &#39;374O&#39;, &#39;URYD&#39;, &#39;M0WG&#39;, &#39;KBWO&#39;, &#39;ONY7&#39;, &#39;VYSA&#39;, &#39;KUPK&#39;, &#39;R7GL&#39;, &#39;HSVE&#39;, &#39;BER4&#39;, &#39;6SKY&#39;, &#39;RH2K&#39;, &#39;ZHC2&#39;, &#39;W3ZV&#39;, &#39;FLXH&#39;, &#39;UYDZ&#39;, &#39;YJXM&#39;, &#39;59QM&#39;]
Not in Train occupation_code  [&#39;0ZND&#39;

In [24]:
train.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,Label
0,4WKQSBB,1/2/2019,F,M,1987,1X1H,2A7I,T4MS,P5DA,4WKQSBB X P5DA,0
1,CP5S02H,1/6/2019,F,M,1981,UAOD,2A7I,T4MS,P5DA,CP5S02H X P5DA,0
2,2YKDILJ,1/6/2013,M,U,1991,748L,QZYX,90QI,P5DA,2YKDILJ X P5DA,0
3,2S9E81J,1/8/2019,M,M,1990,1X1H,BP09,56SI,P5DA,2S9E81J X P5DA,0
4,BHDYVFT,1/8/2019,M,M,1990,748L,NO3L,T4MS,P5DA,BHDYVFT X P5DA,0


In [25]:
# LABEL ENCODE
def encode_LE(train,test,cols,verbose=True):
    for col in cols:
        df_comb = pd.concat([train[col],test[col]],axis=0)
        df_comb,_ = df_comb.factorize(sort=True)
        nm = col
        if df_comb.max()>32000: 
            train[nm] = df_comb[:len(train)].astype('int32')
            test[nm] = df_comb[len(train):].astype('int32')
        else:
            train[nm] = df_comb[:len(train)].astype('int16')
            test[nm] = df_comb[len(train):].astype('int16')
        del df_comb; x=gc.collect()
        if verbose: print(nm,', ',end='')

In [26]:
encode_LE(train, test, ['ID','branch_code', 'occupation_code','occupation_category_code','PCODE','sex','marital_status'])

ID , branch_code , occupation_code , occupation_category_code , PCODE , sex , marital_status , 

In [27]:
train.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,Label
0,5384,1/2/2019,0,2,1987,0,19,5,15,4WKQSBB X P5DA,0
1,13834,1/6/2019,0,2,1981,11,19,5,15,CP5S02H X P5DA,0
2,3217,1/6/2013,1,6,1991,3,185,1,15,2YKDILJ X P5DA,0
3,3010,1/8/2019,1,2,1990,0,82,0,15,2S9E81J X P5DA,0
4,12546,1/8/2019,1,2,1990,3,157,5,15,BHDYVFT X P5DA,0


In [33]:
test.tail()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE
209995,506,1/4/2019,0,2,1986,3,19,5,5,0GMU5UH X ECY3
209996,19909,1/8/2019,1,2,1980,11,3,5,5,I9W11CD X ECY3
209997,4460,1/4/2018,1,2,1993,3,200,0,5,42WTEGT X ECY3
209998,9194,1/5/2015,0,6,1981,2,19,5,5,8EKC4O9 X ECY3
209999,35311,1/11/2018,1,6,1977,4,19,5,5,WHII93K X ECY3


In [34]:
final_train = train.iloc[:, :9]

In [35]:
final_train.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE
0,5384,1/2/2019,0,2,1987,0,19,5,15
1,13834,1/6/2019,0,2,1981,11,19,5,15
2,3217,1/6/2013,1,6,1991,3,185,1,15
3,3010,1/8/2019,1,2,1990,0,82,0,15
4,12546,1/8/2019,1,2,1990,3,157,5,15


In [56]:
final_train.to_csv('final.csv')

In [63]:
final = pd.read_csv('final.csv',index_col='ID')
final.columns

Index([&#39;index&#39;, &#39;join_date&#39;, &#39;sex&#39;, &#39;marital_status&#39;, &#39;birth_year&#39;,
       &#39;branch_code&#39;, &#39;occupation_code&#39;, &#39;occupation_category_code&#39;, &#39;PCODE&#39;],
      dtype=&#39;object&#39;)

In [62]:

final.head()

,index,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE
0,0,5384,01-02-19,0,2,1987,0,19,5,15
1,1,13834,01-06-19,0,2,1981,11,19,5,15
2,2,3217,01-06-13,1,6,1991,3,185,1,15
3,3,3010,01-08-19,1,2,1990,0,82,0,15
4,4,12546,01-08-19,1,2,1990,3,157,5,15


In [66]:
final = final.drop(['index'], axis=1)

In [67]:
final.head()

,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE
ID,,,,,,,,
5384,01-02-19,0,2,1987,0,19,5,15
13834,01-06-19,0,2,1981,11,19,5,15
3217,01-06-13,1,6,1991,3,185,1,15
3010,01-08-19,1,2,1990,0,82,0,15
12546,01-08-19,1,2,1990,3,157,5,15


In [68]:
final.nunique()

join_date                   132
sex                           2
marital_status                8
birth_year                   61
branch_code                  15
occupation_code             233
occupation_category_code      6
PCODE                        21
dtype: int64